In [249]:
#reddit account
#username:IES_Python
#email:95346725@fsv.cuni.cz

In [3]:
#import sys
#!{sys.executable} -m pip install praw
#!{sys.executable} -m pip install pandas-datareader
from bs4 import BeautifulSoup
import requests
import time
from datetime import date
import pandas as pd
import numpy as np

In [24]:
class Yahoo_Extractor:
    def __init__(self):
        pass

    def getData(self, acro):
        link = 'https://finance.yahoo.com/quote/' + acro + '/history?p=' + acro
        r = requests.get(link)
        r.encoding = 'utf-8-sig'
        self.soup = BeautifulSoup(r.text, 'html.parser')
        lst = []
        stocksDF = pd.DataFrame({'Date':pd.Series([], dtype='str'),
                               acro:pd.Series([], dtype='str')})
        for tr in self.soup.findAll('tr', {'class':'BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)'}):
            Date = tr.contents[0].text
            try:
                Volume = tr.contents[6].text.replace(",","")
            except IndexError:
                Volume = 'NA'
            stocks_row = pd.DataFrame({'Date':pd.Series([Date], dtype='str'),
                                     acro:pd.Series([Volume], dtype='str')})
            stocksDF = stocksDF.append(stocks_row)
        return stocksDF
    
    def joinData(self, acro):
        finalDF = pd.DataFrame({'Date':pd.Series([], dtype='str')})
        for a in acro:
            start = time.time()
            stocksDF = self.getData(a)
            #finalDF = pd.concat([finalDF, stocksDF], axis = 1, join = 'outer', copy = False)
            finalDF = pd.merge(finalDF, stocksDF, how = 'outer', on = 'Date', copy = False)
            end = time.time()
            print('Ellapsed time for stock', a)
            print(end-start)
        finalDF = finalDF.drop_duplicates(subset = 'Date') #removing duplicate entries  
        return finalDF
        
        

In [25]:
#acro = ['MMM', 'ABT', 'ABBV', 'ABMD', 'AAPL','AMZN', 'BRK-B'] #several acronyms for test
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
acro = tickers[0]['Symbol'].tolist()             #creating a list of tickers
acro = [a.replace('.', '-') for a in acro]    #converting the wikipedia ticker notation into Yahoo Finance ticker notation

acro250 = acro[:251]
acro500 = acro[252:504]



In [29]:
Y = Yahoo_Extractor()
DF250 = Y.joinData(acro250)
DF500 = Y.joinData(acro500)
finalDF = pd.merge(DF250, DF500, how = 'outer', on = 'Date', sort = True, copy = False)

#Converting the columns to numeric for better memory allocation
finalDF.iloc[:,1:] = finalDF.iloc[:,1:].apply(pd.to_numeric, downcast = "unsigned")
finalDF.memory_usage(deep=True)


Index     800
Date     6900
MMM      3200
ABT      3200
ABBV     3200
         ... 
XYL      3200
YUM      3200
ZBRA     3200
ZBH      3200
ZION     3200
Length: 505, dtype: int64

In [30]:
today = date.today()
stringEpoch = today.strftime("%d_%m_%y")
s = ''
nameVars = ["C:\\Users\\hso20\\Python\\Project\\YahooData\\",stringEpoch,".csv"]
finalDF.to_csv(path_or_buf=s.join(nameVars),sep = ';', index=True, encoding='utf-8-sig')